In [ ]:
%matplotlib notebook
%precision 3

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [9.5, 7]
import datetime
import pandas
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator, LogLocator, FixedLocator, FixedFormatter, NullLocator)
from matplotlib.dates import DateFormatter




In [ ]:
prefix="/home/ahu/git/gazproject/"

In [ ]:
nleng=pandas.read_csv(prefix+"nluse.csv")
nleng[["begin", "end"]] = nleng['MTU'].str.split(' - ', 1, expand=True)
nleng["timestamp"]=pandas.to_datetime(nleng["begin"], format="%d.%m.%Y %H:%M", utc=True)
nleng.sort_values(["timestamp"], inplace=True)
nleng.set_index("timestamp", inplace=True)
nleng=nleng[nleng["Fossil Gas  - Actual Aggregated [MW]"] != "-"]
nleng["gaspower"]=pandas.to_numeric(nleng["Fossil Gas  - Actual Aggregated [MW]"])
nleng["coalpower"]=pandas.to_numeric(nleng["Fossil Hard coal  - Actual Aggregated [MW]"])
nleng["windpower"]=pandas.to_numeric(nleng["Wind Offshore  - Actual Aggregated [MW]"]) + pandas.to_numeric(nleng["Wind Onshore  - Actual Aggregated [MW]"])
nleng["nukepower"]=pandas.to_numeric(nleng["Nuclear  - Actual Aggregated [MW]"])
nleng["wastepower"]=pandas.to_numeric(nleng["Waste  - Actual Aggregated [MW]"])
nleng["otherpower"]=pandas.to_numeric(nleng["Other  - Actual Aggregated [MW]"])
nleng.tail(4)

In [ ]:
def addATransport(df, fname):
    nltransp=pandas.read_csv(prefix+fname)
    print(nltransp.columns[1])
    nltransp[["begin", "end"]] = nltransp['Time (UTC)'].str.split(' - ', 1, expand=True)
    nltransp["timestamp"]=pandas.to_datetime(nltransp["begin"], format="%d.%m.%Y %H:%M", utc=True)
    nltransp.sort_values(["timestamp"], inplace=True)
    nltransp.set_index("timestamp", inplace=True)

    nltransp=nltransp[nltransp[nltransp.columns[1]] != "-"] # filter out empty lines
    nltransp=nltransp[nltransp[nltransp.columns[2]] != "-"] # filter out empty lines
    
    nltransp[nltransp.columns[1]]=pandas.to_numeric(nltransp[nltransp.columns[1]])
    nltransp[nltransp.columns[2]]=pandas.to_numeric(nltransp[nltransp.columns[2]])
    nltransp=nltransp.drop("begin", axis=1)
    nltransp=nltransp.drop("end", axis=1)
    nltransp=nltransp.drop("Time (UTC)", axis=1)
    
    plt.figure()
    plt.title(fname)
    plt.plot(nltransp[nltransp.columns[0]] - nltransp[nltransp.columns[1]])
    nltransp=nltransp.resample("15T").interpolate("time")
    plt.plot(nltransp[nltransp.columns[0]] - nltransp[nltransp.columns[1]], '-+')

    if(len(df)==0):
        print("Copying!")
        df=nltransp
    else:
        df=df.join(nltransp)
    print(len(df))
    return df

transp=pandas.DataFrame()
transp=addATransport(transp, "nlbe.csv")
transp=addATransport(transp, "nldk.csv")
transp=addATransport(transp, "nldelu.csv")
transp=addATransport(transp, "nlgb.csv")
transp=addATransport(transp, "nlno2.csv")
importpower=(transp["BZN|BE > BZN|NL [MW]"] + transp["BZN|DE-LU > BZN|NL [MW]"] + transp["BZN|DK1 > BZN|NL [MW]"] + transp["BZN|GB > BZN|NL [MW]"] + transp["BZN|NO2 > BZN|NL [MW]"]) -          (transp["BZN|NL > BZN|BE [MW]"] + transp["BZN|NL > BZN|DE-LU [MW]"] + transp["BZN|NL > BZN|DK1 [MW]"] + transp["BZN|NL > BZN|GB [MW]"] + transp["BZN|NL > BZN|NO2 [MW]"])
transp


In [ ]:
#print(transp.tail(10))
plt.figure()
plt.plot(transp["BZN|BE > BZN|NL [MW]"] - transp["BZN|NL > BZN|BE [MW]"], label="BE", alpha=0.4)
plt.plot(transp["BZN|DE-LU > BZN|NL [MW]"] - transp["BZN|NL > BZN|DE-LU [MW]"], label="DE-LU", alpha=0.4)
plt.plot(transp["BZN|DK1 > BZN|NL [MW]"] - transp["BZN|NL > BZN|DK1 [MW]"], label="DK1", alpha=0.4)
plt.plot(transp["BZN|GB > BZN|NL [MW]"] - transp["BZN|NL > BZN|GB [MW]"], label="GB", alpha=0.4)
plt.plot(transp["BZN|NO2 > BZN|NL [MW]"] - transp["BZN|NL > BZN|NO2 [MW]"], label="NO2", alpha=0.4)



plt.plot(importpower,
        label="Total")
plt.grid()
plt.legend()
plt.ylabel("MW")

In [ ]:
plt.figure()
#plt.plot(nleng["gaspower"], label="Gas")
#plt.plot(nleng["coalpower"], label="Coal")
#plt.plot(nleng["windpower"], label="Wind")
#plt.plot(nleng["nukepower"], label="Nuclear")
#plt.plot(nleng["wastepower"], label="Waste")
plt.plot(nleng["otherpower"].rolling("1d", center=True).mean(), label="Other")
plt.plot(importpower.rolling("1d", center=True).mean(), label="Import")
plt.grid()
plt.ylabel("MW")
plt.title("ENTSO-E / TENNET Dutch actual electricity production")
plt.legend()

In [ ]:
plt.figure()
labels=["Nuke", "Wind", "Coal", "Gas", "Other"]
plt.stackplot(nleng.index,
              nleng["nukepower"],
                nleng["windpower"],
                nleng["coalpower"],
                nleng["gaspower"], nleng["otherpower"],
              labels=labels
             )

plt.plot((nleng["nukepower"]+nleng["windpower"]+nleng["coalpower"]+nleng["gaspower"]+nleng["otherpower"]).rolling("1h", center=True).mean(), label="Daily averaged sum")
plt.plot((nleng["nukepower"]+nleng["windpower"]+nleng["coalpower"]+nleng["gaspower"]+nleng["otherpower"]+importpower).rolling("1h", center=True).mean(), label="Daily averaged sum imp")

plt.plot(importpower.rolling("1h", center=True).mean())
plt.legend()
plt.grid()
plt.ylabel("MW")
plt.title("Dutch electricity generation by source")

In [ ]:
plt.figure()
#plt.plot(100.0*(nleng["coalpower"]/(nleng["nukepower"]+nleng["windpower"]+nleng["coalpower"]+nleng["gaspower"]+nleng["otherpower"])).rolling("7d", center=True).mean(), label='Coal')
#plt.plot((nleng["coalpower"]).rolling("1d", center=True).mean()/1000.0, label='Coal')
#plt.plot((nleng["coalpower"]).rolling("7d", center=True).mean()/1000.0, label='Coal weekly basis')

#plt.plot((nleng["gaspower"]).rolling("1d", center=True).mean()/1000.0, label='gas')
#plt.plot((nleng["gaspower"]).rolling("7d", center=True).mean()/1000.0, label='gas weekly basis')

plt.plot((nleng["windpower"]).rolling("1d", center=True).mean()/1000.0, label='wind')
plt.plot((nleng["windpower"]).rolling("7d", center=True).mean()/1000.0, label='wind weekly basis')

#plt.ylabel("%")
plt.ylabel("GW")
plt.ylim(0)
plt.title("Local Dutch electrical power generation from various sources")
plt.axvline(datetime.date(2022, 2, 24), ls=':', color='red', label="Start of invasion")

plt.legend()
plt.grid()

In [ ]:
#print(importpower.resample("15T").interpolate("time"))
knownpower=(nleng["nukepower"]+nleng["windpower"]+nleng["coalpower"]+nleng["gaspower"]+nleng["wastepower"]+nleng["otherpower"])+importpower
knownpower=knownpower["2022-03-21":"2022-03-25"]
print(knownpower.head(10))
plt.figure()
#
kpdf=knownpower.to_frame()
kpdf["time"]=pandas.to_datetime(kpdf.index.time.astype(str))
hh_mm = DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(hh_mm)

for dnum in range(21, 26):
    daystr="2022-03-"+str(dnum).zfill(2)
    day=kpdf.loc[daystr]
    plt.plot(day.time, day[0], label=daystr, alpha=0.9)

weekavg=kpdf.groupby(["time"])[0].mean()
#weekavg.plot(label="Average")

interp=pandas.DataFrame()
interp["time"]=[pandas.to_datetime("06:00"), pandas.to_datetime("13:00"), pandas.to_datetime("19:00")]
interp["power"]=[8690, 11500, 10190]
interp.set_index("time", inplace=True)
interp=interp.resample("15T").interpolate("quadratic")
#print(interp)
plt.plot(interp["power"], label="Stupid model of real power use")

plt.plot(interp["power"]-weekavg, label="Imputed solar power production")
print((interp["power"]-weekavg).sum()/4)
plt.ylim(0, 14000)
plt.grid()
plt.legend(loc=2)
plt.ylabel("MW")
plt.title("The Dutch 'duck curve': known electricity generation\n(coal+gas+nuclear+wind+waste+other+import) for national use")
plt.xlabel("UTC")